In [1]:
import pandas as pd


# Chargement des fichiers CSV
benin_data = pd.read_csv("../data/dataBell_DNS Dataset/features_domain_benign_csv.csv")
malware_data = pd.read_csv("../data/Bell_DNS Dataset/features-domain_Malware.csv")
phishing_data = pd.read_csv("../data/Bell_DNS Dataset/features-domain_phishing.csv")
spam_data = pd.read_csv("../data/dataBell_DNS Dataset/features-domain_Spam.csv")

# Ajout de la colonne 'Class' 
benin_data['Class'] = 'Benin'
malware_data['Class'] = 'Malware'
phishing_data['Class'] = 'Phishing'
spam_data['Class'] = 'Spam'


# Concaténation de tous les ensembles de données
all_data = pd.concat([benin_data, malware_data, phishing_data, spam_data], ignore_index=True)


In [14]:
# Spécifier la colonne cible je doit le faire avant l'encodage
y = all_data['Class']

In [15]:
from sklearn.impute import SimpleImputer

# Concaténation de tous les ensembles de données
all_data = pd.concat([benin_data, malware_data, phishing_data, spam_data], ignore_index=True)

# Convertir les variables catégorielles en variables numériques
all_data_encoded = pd.get_dummies(all_data, columns=['Class'], drop_first=True)

# Exclure les colonnes non numériques
X = all_data_encoded.select_dtypes(include='number')

# Vérifier les colonnes avec des valeurs manquantes
missing_columns = X.columns[X.isnull().any()]

print("Missing value : {}", X[missing_columns])

# Imputation des valeurs manquantes
if not missing_columns.empty:
    imputer = SimpleImputer(strategy='mean')  # Vous pouvez également utiliser 'median' ou 'most_frequent'
    X[missing_columns] = imputer.fit_transform(X[missing_columns])



#apres remplissage avec la valuer moyenne    
print("Missing value : {}", X[missing_columns])



Missing value : {}        subdomain  Alexa_Rank  Page_Rank
0            0.0         1.0        NaN
1            0.0         1.0        NaN
2            1.0         1.0        NaN
3            1.0         1.0        NaN
4            0.0         7.0        NaN
...          ...         ...        ...
50154        0.0        -1.0        NaN
50155        0.0        -1.0        NaN
50156        0.0        -1.0        NaN
50157        0.0        -1.0        NaN
50158        0.0        -1.0        NaN

[50159 rows x 3 columns]
Missing value : {}        subdomain  Alexa_Rank  Page_Rank
0            0.0         1.0       -1.0
1            0.0         1.0       -1.0
2            1.0         1.0       -1.0
3            1.0         1.0       -1.0
4            0.0         7.0       -1.0
...          ...         ...        ...
50154        0.0        -1.0       -1.0
50155        0.0        -1.0       -1.0
50156        0.0        -1.0       -1.0
50157        0.0        -1.0       -1.0
50158        0.0

In [16]:
num_features = X.shape[1]
print("Nombre total de caractéristiques numeriques (car les autres ont été ignorer):", num_features)


#donc dans notre fonction il faut passer un nombre de caracteristiques inferieur ou egale a 13

Nombre total de caractéristiques numeriques (car les autres ont été ignorer): 13


In [12]:
########### j'affiche la variance ici 

X.var()

subdomain             2.468767e-01
len                   5.041374e+01
numeric_percentage    4.878897e+01
entropy               3.723222e-01
dec_8                 0.000000e+00
dec_32                6.559620e-03
oc_8                  0.000000e+00
oc_32                 7.974164e-05
hex_8                 0.000000e+00
hex_32                0.000000e+00
puny_coded            4.682994e-03
Alexa_Rank            3.907441e+12
Page_Rank             0.000000e+00
dtype: float64

In [17]:
#comme jai des donnée negative alors "chi2" est pour une selection univariée des caracteristique  qui necessite des valeurs non negative il fait soit trnsformer les donner soit use une autre methode de selection des caracteristique

X_non_negative = X - X.min().min()


In [18]:
############# j'affiche la variance pour voir 

X_non_negative.var()


subdomain             2.468767e-01
len                   5.041374e+01
numeric_percentage    4.878897e+01
entropy               3.723222e-01
dec_8                 0.000000e+00
dec_32                6.559620e-03
oc_8                  0.000000e+00
oc_32                 7.974164e-05
hex_8                 0.000000e+00
hex_32                0.000000e+00
puny_coded            4.682994e-03
Alexa_Rank            3.907441e+12
Page_Rank             0.000000e+00
dtype: float64

In [19]:
import pandas as pd
from sklearn.feature_selection import SelectKBest, chi2

def univariate_selection(X_non_negative, y, feature_number):
    best_features = SelectKBest(score_func=chi2, k=feature_number)
    fit = best_features.fit(X_non_negative, y)
    
    df_scores = pd.DataFrame(fit.scores_)
    df_columns = pd.DataFrame(X_non_negative.columns)
    
    feature_scores = pd.concat([df_columns, df_scores], axis=1)
    feature_scores.columns = ['Features', 'Score']
    
    return feature_scores.nlargest(feature_number, 'Score')['Features'].values



In [20]:
# Appel de la fonction et impression des features importants
important_features_univariate = univariate_selection(X_non_negative, y, feature_number=10)
print("Features importants (Univariate Selection):")
print(important_features_univariate)


Features importants (Univariate Selection):
['Alexa_Rank' 'numeric_percentage' 'len' 'entropy' 'subdomain'
 'puny_coded' 'dec_32' 'oc_32' 'dec_8' 'oc_8']
